In [15]:
import requests
import json
import os
import pandas as pd
import xml.etree.ElementTree as ET

In [16]:
def search_api(root, output_dir, pageSize, url_template):
    # Loop through topics and search for the top 1000 arguments
    for topic in root.findall('topic'):
        topic_number = topic.find('number')
        if topic_number is not None:
            topic_id = topic_number.text
            topic_title = topic.find('title')
            if topic_title is not None and topic_title.text:
                topic_title = topic_title.text.strip()
            else:
                topic_title = ''

            # Define the search query using the topic title
            #query = {
            #    "query": {
            #        "match": {
            #            "text": topic_title
            #        }
            #    },
            #    "size": 1000,
            #    "explain": True
            #}
            
            #print ("{}:{}".format(topic_title, pageSize))
            url = url_template.format(topic_title, pageSize)

            # Execute the search query on Elasticsearch
            #results = es.search(index=index_name, body=query)['hits']['hits']
            response = requests.get(url).json()
            results = response['arguments']
            print (len(results))
            
            # Save the top 1000 arguments for the current topic to a file
            os.makedirs(output_dir, exist_ok=True)
            #with open(os.path.join(output_dir, f"{topic_id}.txt"), "w") as f:
            topic_file_name = os.path.join(output_dir, f"{topic_id}.txt")
            header = ['id', 'conclusion', 'premises', 'stance', 'rank']
            arguments = []
            for i, result in enumerate(results):
                #argument = result['_source']['text']
                argument = [result['id'], result['conclusion'], result['premises'][0]['text'], result['stance'], i + 1]
                arguments.append(argument)
            
            #print (len(arguments))
            #print (len(arguments[0]))
            
            retrieved_arguments_pd = pd.DataFrame(arguments, columns=header)
            retrieved_arguments_pd.to_csv(topic_file_name, sep='\t')
            #break
                #f.write(f"{topic_id} {i+1}\t{argument}\n")  

In [19]:
# Load the XML files
tree21 = ET.parse('/Users/balazs/Desktop/dissertationProjectCode/dissertationCodeBase/Data/topic_files/topics-task-1-only-titles.xml')
root21 = tree21.getroot()

#tree22 = ET.parse('/Users/balazs/Desktop/dissertationProjectCode/dissertationCodeBase/Data/topic_files/topics2022.xml')
#root22 = tree22.getroot()

# Define the output directories
output_dir21 = "/Users/balazs/Desktop/dissertationProjectCode/dissertationCodeBase/Data/args21/"
#output_dir22 = "/Users/balazs/Desktop/dissertationProjectCode/dissertationCodeBase/Data/args22/"

pageSize = 1000
#url_template = "https://args.me/api/v2/arguments?query={}&corpora=args-me-2020-04-01&pageSize={}&format=json"
url_template = "https://args.me/api/v2/arguments?query={}&pageSize={}&format=json"

# Call search functions
search_api(root21, output_dir21, pageSize, url_template)
#search_topics(es, root22, output_dir22, index_name="my-custom-index")

0
0
0
170
0
0
0
0
0
0
802
0
10
0
23
1000
0
0
0
0
0
8
0
62
143
1000
10
14
330
0
0
161
0
21
0
5
2
12
0
2
0
8
0
2
0
0
0
0
0
0


In [18]:
# Print the response
print (len(response))
#print(response)
print (len(response['arguments']))

5
216
